In [3]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset=fetch_20newsgroups(shuffle=True, random_state=1,remove=('headers','footers','puotes'))
documents=dataset.data
len(documents)

11314

In [6]:
documents[0]

"Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n"

In [7]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [8]:
news_df=pd.DataFrame({'document':documents})
news_df['clean_doc']=news_df['document'].str.replace('[^a-zA-Z]',' ')
news_df['clean_doc']=news_df['clean_doc'].apply(lambda x:' '.join([w for w in x.split() if len(w)]))
news_df['clean_doc']=news_df['clean_doc'].apply(lambda x:x.lower())

<ipython-input-8-a602a6154acc>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc']=news_df['document'].str.replace('[^a-zA-Z]',' ')


In [9]:
news_df['clean_doc'][0]

'well i m not sure about the story nad it did seem biased what i disagree with is your statement that the u s media is out to ruin israels reputation that is rediculous the u s media is the most pro israeli media in the world having lived in europe i realize that incidences such as the one described in the letter have occured the u s media as a whole seem to try to ignore them the u s is subsidizing israels existance and the europeans are not at least not to the same degree so i think that might be a reason they report more clearly on the atrocities what is a shame is that in austria daily reports of the inhuman acts commited by israeli soldiers and the blessing received from the government makes some of the holocaust guilt go away after all look how the jews are treating other races when they got power it is unfortunate'

In [10]:
from nltk.corpus import stopwords
stop_words=stopwords.words('english')
tokenized_doc=news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc=tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

In [11]:
print(tokenized_doc[0])

['well', 'sure', 'story', 'nad', 'seem', 'biased', 'disagree', 'statement', 'u', 'media', 'ruin', 'israels', 'reputation', 'rediculous', 'u', 'media', 'pro', 'israeli', 'media', 'world', 'lived', 'europe', 'realize', 'incidences', 'one', 'described', 'letter', 'occured', 'u', 'media', 'whole', 'seem', 'try', 'ignore', 'u', 'subsidizing', 'israels', 'existance', 'europeans', 'least', 'degree', 'think', 'might', 'reason', 'report', 'clearly', 'atrocities', 'shame', 'austria', 'daily', 'reports', 'inhuman', 'acts', 'commited', 'israeli', 'soldiers', 'blessing', 'received', 'government', 'makes', 'holocaust', 'guilt', 'go', 'away', 'look', 'jews', 'treating', 'races', 'got', 'power', 'unfortunate']


In [12]:
detokenized_doc=[]
for i in range(len(news_df)):
    t=' '.join(tokenized_doc[i])
    detokenized_doc.append(t)
news_df['clean_doc']=detokenized_doc

In [13]:
news_df['clean_doc'][0]

'well sure story nad seem biased disagree statement u media ruin israels reputation rediculous u media pro israeli media world lived europe realize incidences one described letter occured u media whole seem try ignore u subsidizing israels existance europeans least degree think might reason report clearly atrocities shame austria daily reports inhuman acts commited israeli soldiers blessing received government makes holocaust guilt go away look jews treating races got power unfortunate'

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer(stop_words='english',max_features=1000)
X=vectorizer.fit_transform(news_df['clean_doc'])
X.shape

(11314, 1000)

In [15]:
from sklearn.decomposition import TruncatedSVD
svd_model=TruncatedSVD(n_components=20)
svd_model.fit(X)
len(svd_model.components_)

20

In [16]:
import numpy as np
np.shape(svd_model.components_)

(20, 1000)

In [17]:
svd_model.components_

array([[ 0.01154736,  0.03865809,  0.03016663, ...,  0.03998349,
         0.01433086,  0.01596249],
       [-0.00251429, -0.03059794, -0.02039921, ...,  0.00668807,
         0.00103059,  0.00947973],
       [-0.01250433, -0.01712072,  0.01899127, ..., -0.01438026,
        -0.00472819, -0.00998351],
       ...,
       [ 0.01038381, -0.0078978 ,  0.05139431, ...,  0.01030142,
        -0.00290781, -0.00738569],
       [-0.00129786, -0.00532508, -0.11627335, ..., -0.00455223,
        -0.03252286, -0.00780538],
       [-0.01128501, -0.01123213, -0.11521741, ...,  0.01017042,
        -0.00231119, -0.00364709]])

In [19]:
terms=vectorizer.get_feature_names()
def get_topics(components,feature_names,n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" (idx+1),[(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(svd_model.components_,terms)

<>:4: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<>:4: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
<ipython-input-19-ce3fef3b77db>:4: SyntaxWarning: 'str' object is not callable; perhaps you missed a comma?
  print("Topic %d:" (idx+1),[(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])


TypeError: 'str' object is not callable